## 1. Tensorflow
### 1.3 Estimator

When using tf.estimator, we need to build two kinds of funcionts. Model and data to be fed.

First, form of model function is like below.

In [ ]:
import tensorflow as tf

##### For Model

In [1]:
def model_fn(features, labels, mode, params, config):
    
    # Build model
    
    return tf.estimator.EstimatorSpec(...)

In this functions,

- ** features: Input data. Tensor or dictionary.

- ** labels: Global true of each data. Tensor or dictionary

- mode: Mode of estimator. Train, Evaluate and Predict.

- params: Hyperparameters we want to apply. Dictionary.

- config: Value for model

After this, we can apply this model function to Estimator object.

In [5]:
# estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = ..., configs= ..., parmas= ...)

model_dir is route to save chekcpoint. And we can implemnet this estimator like,

- For training

estimator.train(input_fn = ...)

- For evaluating

estimator.evaluate(input_fn = ...)

- For predicting

estimator.predict(input_fn = ...)

##### For input data

For function of input data, tf.data is used. 

Data for each situation, train, evaluate or test. is different. So, we define each function respectively.

In [6]:
def train_input_fn():
    
    # Build data pipeline
    
    return features, labels

## Build DNN model for Sentiment Analysis
##### Create data

In [12]:
import tensorflow as tf
from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

labels = [[1], [0], [1], [1], [0], [1]] # It doesn't have meaning.

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index

##### Define train_input_fn 

In [13]:
EPOCH = 100

def train_input_fn():
    
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    dataset = dataset.repeat(EPOCH)
    dataset = dataset.batch(1)
    dataset = dataset.shuffle(len(sequences))
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

##### Define model_fn

In [18]:
VOCAB_SIZE = len(word_index) + 1
EMB_SIZE = 128

def model_fn(features, labels, mode):
    
    # To set each statement be 'True'
    
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    embed_input = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)(features)
    embed_input = tf.reduce_mean(embed_input, axis=1)
    
    hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
    output_layer = tf.keras.layers.Dense(1)(hidden_layer)
    output = tf.nn.sigmoid(output_layer)
    
    loss = tf.losses.mean_squared_error(output, labels)
    
    if TRAIN:
        global_step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode,
                                          train_op=train_op,
                                          loss=loss)

##### Implement

In [19]:
DATA_OUT_PATH = 'C:/python/NLP/Chap_2/data_out'

import os

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir = DATA_OUT_PATH + 'checkpoint/dnn')
estimator.train(train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:/python/NLP/Chap_2/data_outcheckpoint/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A7B4A29E10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSave